In [1]:
import requests
import dotenv
import os
dotenv.load_dotenv()  # .env 환경변수 파일을 로드함

key = os.getenv("SEOUL_SUBWAY_API")

In [2]:
# 지하철 실시간 도착정보  https://data.seoul.go.kr/dataList/OA-12764/F/1/datasetView.do
# http://swopenAPI.seoul.go.kr/api/subway/(인증키)/xml/realtimeStationArrival/0/5/서울

url = f'http://swopenAPI.seoul.go.kr/api/subway/{key}/json/realtimeStationArrival/0/100/강남'
response = requests.get(url)
response.status_code

200

In [3]:
respnse = response.json()
respnse

{'status': 500,
 'code': 'ERROR-338',
 'message': '해당 인증키로는 실시간 서비스를 사용할 수 없습니다.',
 'link': '',
 'developerMessage': '',
 'total': 0}